In [39]:
import matplotlib.pyplot as plt
import os
import cv2 as cv
import numpy as np

Steps:
Convert left view to CIELAB -> lightness is the monochrome image
Right view is the guidance image
Original left view is the ground truth

Guidance image is downsampled to half size
https://en.wikipedia.org/wiki/Bayer_filter
Green channel is "replicated" to have 2x green a red, blue. 
(Check whether RGB to Bayer is the same thing) 
cv2.cvtColor 
https://docs.opencv.org/4.x/db/d8c/group__cudaimgproc__color.html#ga7fb153572b573ebd2d7610fcbe64166e
SceneFlow -> get to later

In [5]:
os.listdir(cur_dir)

['.ipynb_checkpoints',
 'ALL-2views',
 'ALL-2views.zip',
 'data_processing_project.ipynb',
 'SVD_Stewart_Wedin_Theorem.pdf']

In [6]:
os.chdir(cur_dir + '\All-2views')

In [7]:
base_dir = os.getcwd()
base_dir

'C:\\Users\\jpienkny\\Documents\\EECS 556\\Report\\All-2views'

In [8]:
os.listdir()

['Art',
 'Books',
 'Computer',
 'Dolls',
 'Drumsticks',
 'Dwarves',
 'Laundry',
 'Moebius',
 'Reindeer']

Let's use "Art" as a sample. Will make this more seamless down the line.

In [18]:
views_wanted = ['\Art', '\Books', '\Computer']
image_list = [] # List Format: it is n x 2, n is the number of images, [0] is left (Ground Truth), [1] is right (Guidance), [2] is mono
disp_list = [] # Same list format, disparity matrices if we need

cur_dir = base_dir
for view in views_wanted:
    new_dir = cur_dir + view
    os.chdir(new_dir)
    img_left = cv.imread('view1.png')
    assert img_left is not None, "file could not be read, check with os.path.exists()"
    img_right = cv.imread('view5.png')
    assert img_right is not None, "file could not be read, check with os.path.exists()"
    
    mono = cv.cvtColor(img_left, cv.COLOR_BGR2LAB) # Mono image
    
    image_list.append([img_left, img_right, mono[:,:,0]]) # Only take luminance image
    
    disp_left = cv.imread('disp1.png')
    disp_right = cv.imread('disp5.png') # If an image doesn't have a disparity img, will store as None, not fail. 
    disp_list.append([disp_left, disp_right])
    cur_dir = base_dir
    
    


In [11]:
len(image_list)

3

Guidance Image Manipulation: 
Downsample to half size (both dims), then convert to bayer format, then do demosaicing interpolation/upsampling

In [28]:

test_image = image_list[0][1] # Original Guidance Image/Right Image
window_name = 'image'
cv.imshow(window_name, test_image)
cv.waitKey(0)

13

In [29]:
cv.destroyAllWindows()

In [30]:
test_image.shape

(370, 463, 3)

In [31]:
test_half = cv.resize(test_image, (0,0), fx = 0.5, fy = 0.5)

In [32]:
test_half.shape # BGR, need BGGR

(185, 232, 3)

In [34]:
test_half_g = test_half[:,:,1]

In [33]:
cv.imshow(window_name, test_half)
cv.waitKey(0)

-1

In [35]:
test_half_g.shape

(185, 232)

In [62]:
bggr_format = np.zeros([test_half.shape[0] * 2,test_half.shape[1] * 2])

In [63]:
bggr_format.shape

(370, 464)

In [44]:
bggr_format[:,:,0] = test_half[:,:,0] #B
bggr_format[:,:,1] = test_half[:,:,1] #G
bggr_format[:,:,2] = test_half[:,:,1] #G
bggr_format[:,:,3] = test_half[:,:,2] #R

In [45]:
bggr_format.shape

(185, 232, 4)

In [51]:
demosaiced = cv.cvtColor(bggr_format, cv.COLOR_BayerBGGR2BGR)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\demosaicing.cpp:1693: error: (-215:Assertion failed) depth == CV_8U || depth == CV_16U in function 'cv::demosaicing'


In [61]:
B = test_half[:,:,0]
B.shape

(185, 232)

In [60]:
B[0::2].shape

(93, 232)

In [64]:
bggr_format[0::,0::] = test_half[:,:,0]

ValueError: could not broadcast input array from shape (185,232) into shape (370,464)

In [65]:
bggr_format[0::,0::].shape

(370, 464)

Algorithm Skeleton

Input Parameters:

In [ ]:
S = 
Wh = 
Wv = 
N = 4 # Using Figure 4
T = N # Paper Results

In [67]:
L = np.zeros(test_image.shape)
U_a = np.zeros(test_image.shape)
U_b = np.zeros(test_image.shape)